# Bộ Định Giá Sản Phẩm (Tiếp tục)

Một mô hình có thể ước lượng giá trị của một sản phẩm dựa trên mô tả của nó.

## Quá Trình Chọn Lọc Dữ Liệu Phần 2

Hôm nay chúng ta sẽ mở rộng bộ dữ liệu để bao phủ nhiều hơn, và chuẩn bị nó thành một bộ dữ liệu xuất sắc để huấn luyện.

Bộ dữ liệu nằm ở đây:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

Và thư mục chứa tất cả các bộ dữ liệu sản phẩm nằm ở đây:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/tree/main/raw/meta_categories

## Bộ Dữ Liệu Nhẹ (Lite dataset)

Notebook này là một lựa chọn thay thế cho `day2.ipynb`, tạo ra một bộ dữ liệu nhỏ hơn chỉ dành cho các Thiết Bị Gia Dụng, nhằm giúp quá trình huấn luyện nhanh hơn và chi phí thấp hơn. Bạn có thể cần cập nhật tên các notebook trong tương lai để phản ánh rằng bạn đã xây dựng bộ dữ liệu "lite" thay vì bộ dữ liệu đầy đủ.

In [ ]:
# imports

import os
import random
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
import pickle

In [ ]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# And some more imports - with thanks to Alex C for pointing out that I need to do these after the HF login..

from items import Item
from loaders import ItemLoader

In [ ]:
%matplotlib inline

## The ItemLoader code

Hãy xem trong loaders.py – có một số đoạn mã hữu ích giúp chúng ta làm việc dễ dàng hơn.

## Now to SCALE UP

Hãy xem tất cả các bộ dữ liệu (datasets) của tất cả các mặt hàng mà bạn có thể tìm thấy trong một cửa hàng bán lẻ gia dụng lớn – electrical, electronic, office và các category liên quan, nhưng không bao gồm clothes / beauty / books.

In [ ]:
dataset_names = [
    # "Automotive",
    # "Electronics",
    # "Office_Products",
    # "Tools_and_Home_Improvement",
    # "Cell_Phones_and_Accessories",
    # "Toys_and_Games",
    "Appliances",
    # "Musical_Instruments",
]

In [ ]:
items = []
for dataset_name in dataset_names:
    loader = ItemLoader(dataset_name)
    items.extend(loader.load())

In [ ]:
print(f"A grand total of {len(items):,} items")

In [ ]:
# Plot the distribution of token counts again

tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="skyblue", bins=range(0, 300, 10))
plt.show()

In [ ]:
# Plot the distribution of prices

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="blueviolet", bins=range(0, 1000, 10))
plt.show()

# Bộ Dữ Liệu Đã Được Chọn Lọc!

Chúng ta đã tạo ra một bộ dữ liệu xuất sắc.

Hãy thực hiện một số kiểm tra cuối cùng.

In [ ]:
# How does the price vary with the character count of the prompt?

sample = items

sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# Create the scatter plot
plt.figure(figsize=(15, 8))
plt.scatter(sizes, prices, s=0.2, color="red")

# Add labels and title
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# Display the plot
plt.show()

In [ ]:
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
report(sample[50])

## Nhận xét

Một điều thú vị về bộ tokenizer của Llama là mọi số từ 1 đến 999 đều được ánh xạ thành 1 token, tương tự như những gì chúng ta đã thấy với gpt-4o. Điều này lại không đúng với qwen2, gemma và phi3, khi mà tất cả đều ánh xạ từng chữ số riêng lẻ thành token. Điều này hóa ra cũng có chút hữu ích cho dự án của chúng ta, mặc dù nó không phải là một yêu cầu thiết yếu.

# Cuối cùng

Đã đến lúc chia nhỏ dữ liệu của chúng ta thành các bộ dữ liệu huấn luyện, kiểm tra và xác thực.

Thông thường, người ta sử dụng 5%-10% dữ liệu cho mục đích kiểm tra, nhưng thực ra hiện tại chúng ta có nhiều dữ liệu hơn mức cần thiết. Chúng ta sẽ lấy 25.000 điểm dữ liệu cho huấn luyện, và dành ra 2.000 điểm cho kiểm tra, mặc dù sẽ không sử dụng hết tất cả chúng.


In [ ]:
random.seed(42)
random.shuffle(sample)
train = sample[:25_000]
test = sample[25_000:27_000]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
print(train[0].prompt)

In [ ]:
print(test[0].test_prompt())

In [ ]:
# Plot the distribution of prices in the first 250 test points

prices = [float(item.price) for item in test[:250]]
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

# Cuối cùng - tải lên bộ dữ liệu mới của bạn

Chuyển đổi thành prompts và tải lên HuggingFace hub

In [ ]:
train_prompts = [item.prompt for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.test_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
# Create a Dataset from the lists

train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
DATASET_NAME = "kenzytran/lite-data"
dataset.push_to_hub(DATASET_NAME, private=True)

In [ ]:
# One more thing!
# Let's pickle the training and test dataset so we don't have to execute all this code next time!

with open('train_lite.pkl', 'wb') as file:
    pickle.dump(train, file)

with open('test_lite.pkl', 'wb') as file:
    pickle.dump(test, file)

## Việc cần làm cho bạn:

- Nghiên cứu sâu hơn về bộ dữ liệu!
- Xác nhận rằng tokenizer ánh xạ tất cả các giá trị 3 chữ số thành 1 token